# Extract data from .sol files

Use this if you are using the Wilkens function mode 2 to extract <a> loop (phase 0) and <c> loop (phase 1)densities

This workbook is an example of how the extractDataDir function can be used to extract physical paramaters from a folder a .sol files. 

This data frame is then manipulated for plotting data from a synchrotron experiment as an example.

## Import stuff

In [ ]:
import pandas as pd
from src.cmwp_tools import extractDataDir
import matplotlib.pyplot as plt
import mplcursors
%matplotlib qt5

## Extract data from solution files

This code extracts data from CMWP .sol files

In [ ]:
data_df = extractDataDir('/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd/*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1

In [ ]:
data_df = extractDataDir('/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa2/*.sol', calcLoop=False, wavelength=0.018505104)
unique = [9.245, 9.745, 10.245, 10.745, 11.245]
edges = [14.838, 14.836, 14.834, 14.832, 14.830]
dpa_val = 2

In [ ]:
data_df = extractDataDir('/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa05/*.sol', calcLoop=False, wavelength=0.018505104)
unique = [-5.5, -5, -4.5, -4, -3.5]
edges = [14.780, 14.782, 14.782, 14.782, 14.782]
dpa_val = 0.5

## Pull out z and y values from filename

In [ ]:
data_df[['f2', 'y', 'f4', 'zs']] = data_df['filename'].str.split('_',expand=True)               # Split by '_'
data_df[['z', 'no']] = data_df['zs'].str.split('.s',expand=True)                                # Split by '.s'
data_df.drop(['f2', 'filename', 'f4', 'zs', 'no'], axis=1, inplace=True)                        # Clear un-needed columns
data_df['y'] = pd.to_numeric(data_df['y'], downcast="float")
data_df['z'] = pd.to_numeric(data_df['z'], downcast="float")

# Rearrange to put y and z first
cols = data_df.columns.tolist()
data_df=data_df[cols[-2:] + cols[:-2]]

## Calculate depth from edge

This takes a list of all y positions (calculated from the data list above) 
and a list of the corresponsing z positions at the edge of the sample, 
to make a new column called 'depth', which is the distance of the pattern away from the sample edge in microns

In [ ]:
# These are the unique y positions
print(data_df['y'].unique())

In [ ]:
data_df.insert(0, 'depth', data_df['y'])
data_df['depth'].replace(to_replace=unique, value=edges, inplace=True)
data_df['depth'] = data_df['z'] - data_df['depth']
data_df['depth'] = np.round(data_df['depth'] * 1000,1)

In [ ]:
data_df

## Plot

This plots dislocation density, <a> loop fraction, M paramater and lattice paramaters as a function of depth from sample edge

In [ ]:
depth, dpa = np.loadtxt('data/srim.txt')

dpa *= dpa_val

fig, axes = plt.subplots(3, 3, figsize=(16,12))
fig.subplots_adjust(left=0.06, right=0.96, top=0.98, bottom=0.1, hspace=0.3, wspace = 0.35)
axes = axes.ravel()

max_x = 40

# Plot the dpa
for sax in axes:
    sax.set_xlabel('Depth [${\mu}m$]', fontsize=12)
    sec = sax.twinx()
    sec.fill(depth, np.array(dpa), alpha=0.2)
    sec.tick_params(axis='both', which='major', labelsize=10)
    sec.set_xlim(0, max_x)
    sec.set_ylim(0)
    sec.set_ylabel('Dose [dpa]', fontsize=12)
    
    sax.set_zorder(sec.get_zorder()+1)
    sax.patch.set_visible(False);

# Split the lattice param axis
lim_defs=[np.round(np.min(data_df['a_lat'])-0.002,3), np.round(np.max(data_df['a_lat'])+0.002,3), 
          np.round(np.min(data_df['c_lat'])-0.002,3), np.round(np.max(data_df['c_lat'])+0.002,3)]

a_ratio = (lim_defs[1]-lim_defs[0])/((lim_defs[3]-lim_defs[2])+(lim_defs[1]-lim_defs[0]))

pos_orig = axes[6].get_position() # get the original position 
axes = np.append(axes, plt.axes([pos_orig.x0, pos_orig.y0 + (pos_orig.height * (1-a_ratio)),  pos_orig.width, pos_orig.height * a_ratio] ))
axes[9].xaxis.set_ticklabels([])
axes[6].set_position([pos_orig.x0, pos_orig.y0,  pos_orig.width, pos_orig.height * (1-a_ratio)])

# Common settings for all axes
for sax in axes:
    sax.set_xlim(0, max_x)
    sax.tick_params(axis='both', which='major', labelsize=10)
    sax.xaxis.set_ticks_position('both')

#### dislocation density

axes[0].errorbar(x=data_df['depth'], y=data_df['d'], yerr = [data_df['d_neg'],data_df['d_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[0].set_ylabel('<a> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[0].set_ylim(0)

#### m

d_lim = 0.6

axes[1].errorbar(x=data_df['depth'][data_df['d']>d_lim], y=np.exp(2)*data_df['e'][data_df['d']>d_lim], 
                 yerr = np.exp(2)*np.array([data_df['e_neg'][data_df['d']>d_lim],data_df['e_pos'][data_df['d']>d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[1].set_ylabel('$M_{<a>}$', fontsize=12)
axes[1].set_ylim(0)

#### dislocation density

axes[3].errorbar(x=data_df['depth'], y=data_df['d_1'], yerr = [data_df['d_1_neg'],data_df['d_1_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[3].set_ylabel('<c> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[3].set_ylim(0)

#### m

axes[4].errorbar(x=data_df['depth'][data_df['d_1']>d_lim], y=np.exp(2)*data_df['e_1'][data_df['d_1']>d_lim], 
                 yerr = np.exp(2)*np.array([data_df['e_1_neg'][data_df['d_1']>d_lim],data_df['e_1_pos'][data_df['d_1']>d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[4].set_ylabel('$M_{<c>}$', fontsize=12)
axes[4].set_ylim(0)

#### a loop fraction

axes[2].errorbar(x=data_df['depth'], y=data_df['d']/(data_df['d']+data_df['d_1']), 
                 marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[2].set_ylabel('$f_{<a>}$', fontsize=12)
axes[2].set_ylim(0, 1)

#### c loop fraction

axes[5].errorbar(x=data_df['depth'], y=data_df['d_1']/(data_df['d']+data_df['d_1']), 
                 marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[5].set_ylabel('$f_{<c>}$', fontsize=12)
axes[5].set_ylim(0, 1)

#### c paramater

axes[6].errorbar(x=data_df['depth'], y=data_df['c_lat'], yerr=data_df['c_err'], marker='o', c='k', ms=3, lw=0, elinewidth=1, capsize=5)
axes[6].set_ylabel('c [Å]', fontsize=12)
axes[6].set_ylim(lim_defs[2], lim_defs[3])
axes[6].set_yticks(np.arange(lim_defs[2], lim_defs[3], 0.001)[:-1])

#### a paramater

axes[9].errorbar(x=data_df['depth'], y=data_df['a_lat'], yerr=data_df['a_err'], marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[9].set_ylabel('a [Å]', fontsize=12)
axes[9].set_ylim(lim_defs[0], lim_defs[1])
axes[9].set_yticks(np.arange(lim_defs[0], lim_defs[1], 0.001))

#### c/a

axes[7].errorbar(x=data_df['depth'], y=data_df['covera'], yerr=data_df['covera_err'], marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[7].set_ylabel('c/a', fontsize=12)

#### cell volume

axes[8].errorbar(x=data_df['depth'], y=data_df['cellvol'], yerr=data_df['cellvol_err'], c='k', marker='o', ms=3, lw=0, elinewidth=1, capsize=5)
axes[8].set_ylabel('Cell volume [$Å^3$]', fontsize=12);

#plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/'+name+'_ellip'+str(ellip)+'.pdf', dpi=600)
#plt.show()

for ax in axes:
    crs = mplcursors.cursor(ax,hover=False)

    crs.connect("add", lambda sel: sel.annotation.set_text(
        'y={:.3f}, z={:.3f}'.format(data_df['y'][int(sel.index)], data_df['z'][int(sel.index)])))
    plt.show()